In [10]:
import os
from pathlib import Path
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Define your dataset directories
image_path = r"C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food"
print("Image path:", image_path)

walk_through_dir(image_path)


from pathlib import Path
image_path = Path(r"C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food")


train_dir = image_path / "train"
test_dir = image_path / "test"



print("train data -", train_dir, "  test data -", test_dir)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

Image path: C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food
There are 2 directories and 0 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food'.
There are 3 directories and 0 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food\test'.
There are 0 directories and 46 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food\test\pizza'.
There are 0 directories and 58 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food\test\steak'.
There are 0 directories and 46 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\food\test\sushi'.
There are 3 directories and 1 images in 'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\dataset\foo

In [11]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import transforms


# Define transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Load datasets
train_data_simple = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_data_simple = datasets.ImageFolder(root=test_dir, transform=test_transforms)

# Define dataloaders
BATCH_SIZE = 50
NUM_WORKERS = 16

train_dataloader_simple = DataLoader(dataset=train_data_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader_simple = DataLoader(dataset=test_data_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Confirm dataloaders are set up correctly
print(f"Train DataLoader has {len(train_dataloader_simple)} batches")
print(f"Test DataLoader has {len(test_dataloader_simple)} batches")


Train DataLoader has 9 batches
Test DataLoader has 3 batches


In [13]:
output_shape=len(train_data_simple.classes)
output_shape

3

In [14]:
def accuracy_fn(y_true, y_pred):
    if len(y_pred.shape) == 1:
        # If y_pred is a 1D tensor, find the maximum value along the first (and only) dimension
        _, predicted = torch.max(y_pred, dim=0)
    else:
        # If y_pred is a 2D tensor, find the maximum value along the second dimension
        _, predicted = torch.max(y_pred, dim=1)
    correct = (predicted == y_true).sum().item()
    accuracy = correct / y_true.shape[0]
    return accuracy



In [15]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class TinyVGG(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
       
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units, out_features=3),
        )

    def forward(self, x: torch.Tensor):
      #  print("Input shape:", x.shape)
        x = self.conv_block_1(x)
      #  print("After conv_block_1:", x.shape)
        x = self.conv_block_2(x)
       # print("After conv_block_2:", x.shape)
        x = self.conv_block_3(x)
       # print("After conv_block_3:", x.shape)
        x = self.conv_block_4(x)
        x = self.adaptive_pool(x)
        #print("After adaptive_pool:", x.shape)
        x = torch.flatten(x, start_dim=1)  # Flatten the tensor except for batch dimension
        #print("After flattening:", x.shape)
        x = self.classifier(x)
        #print("Output shape:", x.shape)
        return x



In [16]:
output_shape

3

In [20]:
model= TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                 hidden_units=100,
                 output_shape=3).to(device)
num_epochs=200
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

n_total_steps = len(train_dataloader_simple)
for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (images, labels) in enumerate(train_dataloader_simple):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './pizza_stek_sushi.pth'
torch.save(model.state_dict(), PATH)

[1] loss: 1.099
[2] loss: 1.098
[3] loss: 1.098
[4] loss: 1.096
[5] loss: 1.092
[6] loss: 1.080
[7] loss: 1.050
[8] loss: 1.023
[9] loss: 1.015
[10] loss: 1.009
[11] loss: 1.013
[12] loss: 1.008
[13] loss: 0.991
[14] loss: 0.986
[15] loss: 1.018
[16] loss: 1.027
[17] loss: 0.997
[18] loss: 0.989
[19] loss: 0.971
[20] loss: 0.985
[21] loss: 0.990
[22] loss: 0.989
[23] loss: 1.006
[24] loss: 1.013
[25] loss: 0.995
[26] loss: 0.978
[27] loss: 0.961
[28] loss: 0.978
[29] loss: 0.952
[30] loss: 0.964
[31] loss: 0.962
[32] loss: 0.963
[33] loss: 0.967
[34] loss: 0.957
[35] loss: 0.953
[36] loss: 0.956
[37] loss: 0.962
[38] loss: 0.932
[39] loss: 0.942
[40] loss: 0.920
[41] loss: 0.929
[42] loss: 0.940
[43] loss: 0.920
[44] loss: 0.926
[45] loss: 0.922
[46] loss: 0.931
[47] loss: 0.922
[48] loss: 0.937
[49] loss: 0.914
[50] loss: 0.920
[51] loss: 0.902
[52] loss: 0.912
[53] loss: 0.909
[54] loss: 0.916
[55] loss: 0.935
[56] loss: 0.914
[57] loss: 0.895
[58] loss: 0.906
[59] loss: 0.882
[60] l

In [18]:
output_shape=len(test_data_simple.classes)
output_shape

3

In [21]:
PATH = r'C:\Users\ashmi\Documents\Artificial Intelligence\Deep Learning\Pytorch\Learning Codes\Deep learning\PRACTICE\pizza_stek_sushi.pth'
# Load the model
model= TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                 hidden_units=100,
                 output_shape=len(train_data_simple.classes)).to(device)
model.load_state_dict(torch.load(PATH))
model.to(device)

# Set the model to evaluation mode
model.eval()

# Calculate accuracy
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_dataloader_simple.dataset)
    for images, labels in test_dataloader_simple:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()
    
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the loaded model: {acc} %')

Accuracy of the loaded model: 73.33333333333333 %
